# Python语言与经济大数据分析

# 第4讲 文本分析入门

#郭峰（上海财经大学投资系副教授）
#Email：guo.feng@mail.sufe.edu.cn
#2019-01-21
#本讲目录
#4.1 文本字数统计
#4.2 文档格式转换(doc\docx转换成txt）
#4.3 字频统计
#4.4 关键词的提取与统计
#4.5 文本编码格式
#4.6 jieba分词

# 4.1 文本字数统计

In [ ]:
#txt字数统计，一个文件
path='D:/python/郭峰Python讲义/数据与结果/'
f= open(path+"政府工作报告2018.txt","r")
report=f.read()  
report_len = len(report)                
line_num = report.count("\n")       #统计行数
words_num = len(report.split())     #统计单词数,这个适合于英文文本，这里强行应用而已，将字符串按空格分割成一个list，统计list的长度
print("字符数：%s,\n行数:%s,\n单词数量%s"%(report_len,line_num,words_num))
# 注：这样统计后得到的数字比word中显示的字数要多，估计是数字、英文字母的意思，word文本显示的字数，2010被算作1次

In [ ]:
#word文档字数统计,一个文件
import docx

path='D:/python/郭峰Python讲义/数据与结果/'
wordtest=docx.Document(path+"政府工作报告2018.docx")  

a=0
for para in wordtest.paragraphs:   
    a=a+len(para.text)            #目前只会按照段落一段段求字数，不知道是否有直接求字数的命令
print(a)

#这个统计出来的字数和上面那个txt文档又有所不同

In [ ]:
#字数统计,文件夹批量处理，txt格式
import os
path = "D:/python/郭峰Python讲义/数据与结果/政府工作报告/" #文件夹目录  
path2 = "D:/python/郭峰Python讲义/数据与结果/" #结果目录  
files= os.listdir(path)  #得到文件夹下的所有文件名称  
wnum={}   #定义一个空字典
for file in files:    #遍历文件夹
     #print(file)
     if not os.path.isdir(file):      #判断是否是文件夹，不是文件夹才打开  
          f = open(path+file,encoding='gbk',errors="replace")     #打开文件  
          try:
                text=f.read()
                file=file[:-4]
                wnum[file]=len(text)     #一个字典
          except:                        #try, except是查验错误的一种方式，正确的话就执行，错误的话，输出到底哪个错误了
                print(file)
#print(wnum)
f = open(path2+"city_wordnum.txt", "w+")  #结果读入到一个txt文件当中
f.write("city"+" "+"year"+" "+"wnum"+"\n")  #列名
for key in wnum:
    f.write(key[:-4]+" "+key[-4:]+' '+str(wnum[key])+"\n")
f.close()

In [ ]:
#中级步骤解析
a='伊春2015.txt'
b=a[:-4]
print(b)
wnum={}
wnum[b]=19500
print(wnum)

In [ ]:
#中间步骤解析
path2 = "D:/python/郭峰Python讲义/数据与结果/" #结果目录  
name =['a1','a2','a3']
seq=['seq11111','seqs22222','seq33333']
f = open(path2+"test.txt", "w+")
f.write("name\tseq\n")   #\t为制表符
for i in range(0, len(name)):
    f.write(name[i] + "\t" + seq[i] + "\n")
f.close()

In [ ]:
#用word进行字数统计,文件夹批量处理
import os
import docx

path = "D:/python/郭峰Python讲义/数据与结果/政府工作报告word/" #文件夹目录  
path2 = "D:/python/郭峰Python讲义/数据与结果/" #结果目录  

files= os.listdir(path)  #得到文件夹下的所有文件名称  
wnum={}
for file in files:    #遍历文件夹  
     if not os.path.isdir(file):      #判断是否是文件夹，不是文件夹才打开  
          f = open(path+file)     #打开文件  
          wordtest=docx.Document(path+file)    
          a=0
          file=file[:-5]
          for para in wordtest.paragraphs:  #按段落统计字数
                a=a+len(para.text)
          else:
                wnum[file]=a
print(wnum)

f = open(path2+"city_wordnum2.txt", "w+")  #结果读入到一个txt文件当中
f.write("city"+" wnum"+"\n")
for key in wnum:
    f.write(key+" "+str(wnum[key])+"\n")
f.close()

# 4.2 格式转换

In [ ]:
#为了读取doc文件，需要安装pywin32模块，非pip安装，需要下载
#安装方式参考了下面两份网帖：
#下载pywin32-221.win-amd64-py3.6， https://jingyan.baidu.com/article/bad08e1ed173d409c85121f8.html
#安装过程出现bug，解决办法（部分参考，问题比帖子简单）：https://www.cnblogs.com/dbass/p/7988930.html
#下面的程序用来安装中出现的注册表不匹配问题（反正我也看不懂）
import sys

from winreg import *

# tweak as necessary
version = sys.version[:3]
installpath = sys.prefix

regpath = "SOFTWARE\\Python\\Pythoncore\\%s\\" % (version)
installkey = "InstallPath"
pythonkey = "PythonPath"
pythonpath = "%s;%s\\Lib\\;%s\\DLLs\\" % (
    installpath, installpath, installpath
)

def RegisterPy():
    try:
        reg = OpenKey(HKEY_CURRENT_USER, regpath)
    except EnvironmentError as e:
        try:
            reg = CreateKey(HKEY_CURRENT_USER, regpath)
            SetValue(reg, installkey, REG_SZ, installpath)
            SetValue(reg, pythonkey, REG_SZ, pythonpath)
            CloseKey(reg)
        except:
            print("*** Unable to register!")
            return
        print("--- Python", version, "is now registered!")
        return
    if (QueryValue(reg, installkey) == installpath and
                QueryValue(reg, pythonkey) == pythonpath):
        CloseKey(reg)
        print("=== Python", version, "is already registered!")
        return
    CloseKey(reg)
    print("*** Unable to register!")
    print("*** You probably have another Python installation!")

if __name__ == "__main__":
    RegisterPy()

In [ ]:
#将一个doc文档转换成docx文档
import sys
import pickle
import re    #这个是正则表达式
import codecs
import string
import shutil
from win32com import client as wc
path = "D:/python/郭峰Python讲义/数据与结果/" #文件夹目录  
word = wc.Dispatch('Word.Application')
doc = word.Documents.Open(path+'定西2011.doc')  
doc.SaveAs(path+'定西2011.docx', 12)  # 转化后路径下的文件    
doc.Close()
word.Quit()
#本程序参考了网贴：https://blog.csdn.net/X21214054/article/details/78873338

In [ ]:
#【实战操作】将一个文件夹doc文档批量转换成ocx文档
import os    
import docx
import sys
import pickle
import re
import codecs
import string
import shutil
from win32com import client as wc
path = "D:/python/郭峰Python讲义/数据与结果/doc/" #文件夹目录  
pathn= "D:/python/郭峰Python讲义/数据与结果/docx/" #文件夹目录
files= os.listdir(path)  #得到文件夹下的所有文件名称  
for file in files:
    word = wc.Dispatch('Word.Application')
#    print(path+'/'+file)
    doc = word.Documents.Open(path+'/'+file)
    doc.SaveAs(pathn+'/'+file[:-4]+'.docx', 12)  # 转化后路径下的文件,
    doc.Close()

In [ ]:
#【实战操作】：docx文件批量转成txt文件【存在部分格式不对的少量文档】
import os
import docx

#教学举例
path = "D:/python/郭峰Python讲义/数据与结果/政府工作报告word/" #文件夹目录  
pathn= "D:/python/郭峰Python讲义/数据与结果/政府工作报告txt/" #文件夹目录

#实际工作
#path = "D:/python/gov_report_prov/省级汇总2006_2017/" #文件夹目录  
#pathn= "D:/python/gov_report_prov/省级汇总2006_2017转txt/" #文件夹目录  

#path = "D:/python/gov_report_city/城市政府工作报告word/" #文件夹目录  
#pathn= "D:/python/gov_report_city/城市政府工作报告转txt/" #文件夹目录  

files= os.listdir(path)  #得到文件夹下的所有文件名称  
for file in files:    #遍历文件夹  
    f = open(path+file)     #打开文件  
    wordtest=docx.Document(path+file)    
    fname=file[:-5]
    for para in wordtest.paragraphs:  #按段落阅读和另存
        try:
            fn = open(pathn+fname+".txt", "a+",encoding='gbk')  #统计结果读入到一个txt文件当中
            fn.write(para.text+'\n')  #要换行
        except:
            fn = open(pathn+fname+".txt", "a+",encoding='utf-8')  #统计结果读入到一个txt文件当中
            fn.write(para.text+'\n')
          
    fn.close()
    f.close()
    

# 4.3 字频统计

In [ ]:
#字频统计

#先定义一个函数，然后进行调用，编程经验丰富者的常用方法
def count_chinese_word(filepath, encoding='gb2312'):
    word_num = {}    
    with open (filepath, 'r', encoding= encoding ,errors="ignore") as txt_file:
            for uchar in txt_file.read():
                if is_chinese(uchar):
                    if uchar in word_num:
                        word_num[uchar] = word_num[uchar] + 1
                    else:
                        word_num[uchar] = 1
    return word_num


#上面的函数调用下面的函数，下面这个函数用来判断是否汉字
#根据汉字编码在统一编码中所占的区间来标识，阿拉伯数字被排除在外了。

def is_chinese(uchar):   
    if uchar >= u'\u4E00' and uchar <= u'\u9FA5':
        return True
    else:
        return False
    
path= "D:/python/郭峰Python讲义/数据与结果/" 
d=count_chinese_word(path+'政府工作报告2018.txt')
d=sorted(d.items(),key=lambda t:t[1], reverse=True)
print(d)

In [ ]:
#比较两份报告字数差异，来自周耿的讲义。

#先定义一个函数，然后进行调用，编程经验丰富者的常用方法
def count_chinese_word(filepath, encoding='gb2312'):
    word_num = {}    
    with open (filepath, 'r', encoding= encoding ,errors="ignore") as txt_file:
            for uchar in txt_file.read():
                if is_chinese(uchar):
                    if uchar in word_num:
                        word_num[uchar] = word_num[uchar] + 1
                    else:
                        word_num[uchar] = 1
    return word_num


#上面的函数调用下面的函数，下面这个函数用来判断是否汉字
#根据汉字编码在统一编码中所占的区间来标识，阿拉伯数字被排除在外了。

def is_chinese(uchar):   
    if uchar >= u'\u4E00' and uchar <= u'\u9FA5':
        return True
    else:
        return False
        
path= "D:/python/郭峰Python讲义/数据与结果/" 
dict1 = count_chinese_word(path+'000001_2016.txt', 'gbk')
dict2=sorted(dict1.items(), key=lambda d: d[1], reverse=True) 
L=float(sum(dict1.values()))
print('平安银行16年报总字数:', L)

a1=[]
a2=[]
for (k_1,v_1) in dict2[0:100]:
    a1.append(k_1)
    a2.append(v_1)
a=set(a1)  

b1=[]
b2=[] 
dict3 = count_chinese_word(path+'000002_2016.txt', 'gbk')
dict4=sorted(dict3.items(), key=lambda d: d[1], reverse=True) 
M=float(sum(dict3.values()))
print ('万科A16年报总字数:', M)
for (k_2,v_2) in dict4[0:100]:
    b1.append(k_2)
    b2.append(v_2)
b=set(b1)
print ('平安银行和万科A2016年报前100高频字中的相同部分及其分别频率:' )   
c=a&b   #求交集
for k in c:
    print(k,float(dict1[k]/L),float(dict3[k]/M))


# 4.4 关键词提取和统计

In [ ]:
#【实战操作】：批量关键词数量统计,命令块格式，文件docx格式，输出一个txt文件，有近200来个样本存在问题
import os
import docx
path= "D:/python/郭峰Python讲义/数据与结果/政府工作报告word/"
pathn= "D:/python/郭峰Python讲义/数据与结果/" 
keyword='债'                  #想统计什么在这里改动就行了,可以改成两个词的
files= os.listdir(path)  #得到文件夹下的所有文件名称  
keyword_num={}
for file in files: 
    f = open(path+file) 
    wordtest=docx.Document(path+file)
    a=0
    file=file[:-5]
    for para in wordtest.paragraphs:  #按段落统计字数,不会全文。。。
        a=a+para.text.count(keyword)
    else:
        keyword_num[file]=a

fn = open(pathn+"citybaogao_keyword_num.txt", "w+")  #统计结果读入到一个txt文件当中
fn.write("city_year"+" "+keyword+"_num"+"\n")
for key in keyword_num:
    f.write(key+" "+str(keyword_num[key])+"\n")
f.close()

In [ ]:
#【实战操作】：批量关键词数量统计,命令块格式，输入txt，输出一个txt文件

import os

#教学案例
#path= "D:/python/郭峰Python讲义/数据与结果/政府工作报告/"
#pathn= "D:/python/郭峰Python讲义/数据与结果/" 

#实际工作
#path = "D:/python/gov_report_city/城市政府工作报告2006_2018txt/" #文件夹目录
#pathn = "D:/python/gov_report_city/" #文件夹目录

keyword='房'                  #想统计什么在这里改动就行了
fn = open(pathn+"citybaogao_keyword_num2.txt", "w+",encoding='gbk',errors='replace')  #统计结果读入到一个txt文件当中
fn.write("city_name"+" "+"year"+" "+"num"+"\n")
files= os.listdir(path)  #得到文件夹下的所有文件名称  

keyword_num=0  #没有再定义成一个字典，直接将结果存储到新的txt文档当中
for file in files: 
    f = open(path+file,encoding='gbk',errors='replace') 
    try:
        text=f.read()
        file=file[:-4]
        wordnum=text.count(keyword)
        fn.write(str(file[:-4])+" "+str(file[-4:])+" "+str(wordnum)+"\n")
        keyword_num=keyword_num+wordnum  #统计这个词累计出现了多少次
    except:
        print(file)
    f.close()
fn.close()
print(keyword_num)

In [ ]:
a='这两天雾霾很严重'
b=a.index('雾')
print(a[a.index('雾'):a.index('雾')+len('雾霾')])

In [ ]:
#【实战操作】：关键词前后一定字数提取，命令块格式，文件docx格式，输出一个txt文件
import os
import docx

#教学案例
path= "D:/python/郭峰Python讲义/数据与结果/政府工作报告word/"
pathn= "D:/python/郭峰Python讲义/数据与结果/" 

#实际工作
#path = "D:/python/gov_report_city/城市政府工作报告word/" #文件夹目录
#pathn = "D:/python/gov_report_city/" #文件夹目录

keyword='债'                  #想统计什么在这里改动就行了，也可以改造成函数形式
wn=10  #关键词前后10个字提取出来
files= os.listdir(path)  #得到文件夹下的所有文件名称  
keyword_nearby={}
for file in files: 
    f = open(path+"/"+file) 
    wordtest=docx.Document(path+"/"+file)
    a=""
    file=file[:-5]
    for para in wordtest.paragraphs:  #按段落统计字数,不会全文。。。
        for item in enumerate(para.text):
#            para.text.replace(u'\xa0', u' ') 
            if item[1] == keyword:
                _index=item[0] 
                a=a+"   "+para.text[_index-wn:_index+wn+1]
    else:
#        a.encode('gbk','ignore')
         a=a.replace(u'\xa0', u' ')
         a=a.replace(u'\ue004',u'')
         keyword_nearby[file]=a
f = open(pathn+"citybaogao_keyword_nearby.txt", "w+")  #统计结果读入到一个txt文件当中
f.write("city_year"+" "+keyword+"_nearby"+"\n")
for key in keyword_nearby:
    f.write(key+" "+str(keyword_nearby[key])+"\n")
f.close()

#注：
#这个程序处理了一个 'gbk' codec can't encode character '\xa0' in position 22，解决过程非常麻烦
#尝试了 f = open("citybaogao_keyword_nearby.txt", "w+",encoding='utf-8')方法，失败
#尝试了 a.encode('gbk','ignore')的方法，失败
#最终使用a=a.replace(u'\xa0', u' ') 的方法将问题解决，但导致输出的文档中，存在少量的乱码
#更重要的是，涉及到大量的不识别字符，总不能一一替代。
#编码问题是一个非常讨厌的问题

In [ ]:
#【实例操作】2018年政府工作报告，删除无用的东西
import os    
import sys
import pickle
import re
import codecs
import string
import shutil
path='D:/python/gov_report_prov/2018/' #2018年原始文件存在即是utf8的格式，又是gbk格式
pathn='D:/python/gov_report_prov/2018new/'
files= os.listdir(path)  #得到文件夹下的所有文件名称
#print(files)  #这是一个列表

for file in files:
    f=open(path+str(file),encoding='gbk')
    try:
        text=f.read()   #个别文件存在乱码 ，所以跳过去
    except:
        continue
    text=text.replace('<span style="font-size: 14px;">','')
    fn=open(pathn+str(file),'w+',encoding='utf-8')
    fn.write(text)
    f.close()
    os.remove(path+str(file))  #修改一个，删除一个，这样留下来的文件就好弄了
    fn.close()

In [ ]:
#【实战操作】挑出一系列关键词所在的句子,不重复出现。
import re
import os

#教学案例
path= "D:/python/郭峰Python讲义/数据与结果/政府工作报告/"
pathn= "D:/python/郭峰Python讲义/数据与结果/" 

#实际工作
#path = "D:/python/gov_report_city/城市政府工作报告2006_2018txt/" #文件夹目录
#pathn = "D:/python/gov_report_city/" #文件夹目录

#keywords=['债','城投','融资平台'] #城投债论文
keywords=['金融','信贷','贷款','融资','债券','集资','借贷','上市','银行','证券','信托','私募','股权基金'] #学生毕业论文
length=len(keywords)
keywordnum=0
fn = open(pathn+"keywordnearby.txt", "w+",encoding='gbk',errors='replace') 
files= os.listdir(path)  #得到文件夹下的所有文件名称  
for file in files: 
    fn.write(str('\n'+file[:-8]+'*'+file[-8:-4]+'*'))
    f = open(path+file,'r',encoding='gbk',errors='replace') 
    text=f.read()
    g = re.split(r"[，。？；,.]", text) #根据文章中的实际情况添加分割句子的标点，这是一个正则表达式
    for line in g:
        for i in range(length):
            if re.match('.*'+keywords[i]+'.*',line):    #这也是一个正则表达式
                if i==0:
                    line=line.replace('\n','')  #删除换行
                    line=line.strip()  #删除空格
                    fn.write(line+'*')
                    keywordnum=keywordnum+1
                else:
                    continue
                for j in range(i):
                    if re.match('.*'+keywords[j]+'.*',line):  #这是为了避免重复统计
                        continue
                    else:
                        line=line.replace('\n','')  #删除换行
                        line=line.strip()  #删除空格
                        fn.write(line+'*')
                        keywordnum=keywordnum+1
            else:
                continue
    f.close()
fn.close()
print(keywordnum)

# 4.5  文本编码格式

In [9]:
#字符编码（Character Encoding）是将字符集中的字符码映射为字节流的一种具体实现方案
#常见的字符编码有 ASCII 编码、UTF-8 编码、GBK 编码等。
#gbk或gbk2312可以用来处理中文
#我一般将编码格式设置为utf-8
#不同电脑默认的编码方式可能不同，所以有时候需要调整
#关于python编码的详细介绍:https://www.cnblogs.com/Jedore/p/7052994.html

#【实际应用】批量修改文件的编码格式
import os 
path='D:/python/郭峰Python讲义/数据与结果/省级政府工作报告2018编码混杂/'
path2='D:/python/郭峰Python讲义/数据与结果/省级政府工作报告2018编码清理/'

files= os.listdir(path)
for file in files:
    oldpath=os.path.join(path,file)
    try:
        f1=open(path+file, 'r',encoding='utf8')  
        content = f1.read()
    except:
        #continue  #通过添加这个continue可以不执行下面两行命令，进而看到到底是哪些文件是utf8编码的
        f1=open(path+file,'r')
        content = f1.read()
    print(content[0:5])
    f2=open(path2+file,'w',encoding='utf8')
    f2.write(content)
    f1.close()
    f2.close()
#其他批量调整的方法：
#https://blog.csdn.net/oh5W6HinUg43JvRhhB/article/details/80681661

政府工作报
（云南省）
2018年
（北京市）
2018年
（四川省）
天津市20
（宁夏）2
（安徽省）
（山东省）
（山西省）
（广东省）
广西壮族自
（新疆）2
（江苏省）
2018年
（河北省）
（河南省）
（浙江省）
（海南省）
政府工作报
2018年
（甘肃省）
（福建省）
2018西
（贵州省）
（辽宁省）
2018年
（陕西省）
（青海省）
（黑龙江）


# 4.6 jieba中文分词

In [ ]:
# jieba 模块的安装 pip模式,WinPython Command Prompt,管理员模式打开， pip install jieba 
# jieba的安装费了非常大的力气，最终换了一个安装盘，从c改成d，居然问题就解决了，到底啥原因也没搞明白
# jieba支持多种分词方式，具体对比如下
import jieba 
seg1 = jieba.cut("我来到南京大学上课") # 默认精确模式，这个用来统计词频率
seg2 = jieba.cut_for_search("我来到南京大学上课") #搜索引擎模式
seg3 = jieba.cut("我来到南京大学上课", cut_all=True) # 全模式
print(','.join(seg1))
print(','.join(seg2))
print(','.join(seg3))

In [ ]:
#分析一份报告出现的中文词汇频率，包含数字、标点等
import jieba
from collections import Counter
path= "D:/python/郭峰Python讲义/数据与结果/" 
with open (path+"000002_2016.txt", 'r', encoding= 'gbk' ,errors="ignore") as file:
    data=file.read()
cut = jieba.cut(data)
data = dict(Counter(cut))
y=sorted(data.items(), key=lambda d:d[1], reverse = True )
print(y)
print(len(y))

In [ ]:
#分析两份报告的中文词汇频率的交集，即包含的共同高频词汇，剔除了数字、标点，以及一个单字词
import jieba
from collections import Counter
path= "D:/python/郭峰Python讲义/数据与结果/" 

with open (path+"000001_2016.txt", 'r', encoding= 'gbk' ,errors="ignore") as file:
    data=file.read()
cut = jieba.cut(data)
data1 = dict(Counter(cut))
data11=data1.copy()
for i in data1:
    if len(i)<2 or data1[i]<10:
        data11.pop(i)
with open (path+"000002_2016.txt", 'r', encoding= 'gbk' ,errors="ignore") as file:
    data=file.read()
cut = jieba.cut(data)
data2 = dict(Counter(cut))
data22=data2.copy()
for i in data2:
    if len(i)<2 or data2[i]<10:
        data22.pop(i)
k1=data11.keys()
k2=data22.keys()
k3=k1&k2
print(k3)

In [ ]:
#网上找到一个，可以求一个文本的“摘要”，还不知道能在什么上面可以用到
#先要安装：pip install snownlp
from snownlp import SnowNLP

text1 = u"美媒称，鉴于全球石油市场过度供给的情况，中国原油需求下滑是其首要担忧之一。过量生产拉低了石油价格，但是中国过去一年左右的疲弱需求引发了缓慢的回弹。"

s = SnowNLP(text1)

print(s.summary(3))

In [ ]:
#【实战操作】将“省*年份”报告中关于环保的表述片段合成一个文档，然后统计词频，并标注词性
import jieba 
import os
from collections import Counter
import jieba.posseg as pseg
import pandas as pd

#实际工作
path = "D:/python/gov_report_prov/data_huanbao/" #文件夹目录
pathn = "D:/python/gov_report_prov/" 

files= os.listdir(path)  #得到文件夹下的所有文件名称  

f_total = open(pathn+"huanbao_total.txt", "w+",encoding='gbk',errors='replace')  #统计结果读入到一个txt文件当中
for file in files: 
    if int(file[-8:-4])<2008:  #2008年后和2008年年前的文件编码格式不一样
        f = open(path+file,encoding='gbk',errors='replace') 
    else:
        f = open(path+file,encoding='utf-8',errors='replace') 
    try:
        text=f.read()
        text=text.replace('\n','')
        text=text.replace(' ','')
        f_total.write(file[0:-4]+'\n')
        f_total.write(text+"\n")
    except:
        print(file)
    f.close()
f_total.close()

word_list=[]
flag_list=[]

df1=pd.DataFrame(columns=['word','type']) 

f_total = open(pathn+"huanbao_total.txt", "r",encoding='gbk',errors='replace') 
text=f_total.read()
words = pseg.cut(text) # 默认精确模式，这个用来统计词频率
for word,flag in words:
    word_list.append(word)
    flag_list.append(flag)  #标注词性
    
df1['word']=word_list 
df1['type']=flag_list
df3=df1.groupby(['word','type']).size()
df3.to_csv(pathn+"huanbao_cut.csv")

In [ ]:
#【实战操作】统计文档字数，对某些重要的单词，进行赋值加权,应用在环保口号研究中
import re
import os

#实际工作
path = "D:/python/gov_report_prov/data_huanbao/" #文件夹目录
pathn = "D:/python/gov_report_prov/" 


fq1 = open(pathn+'赋权1.txt',encoding='gbk',errors='replace')
fq1=fq1.read()
keyword1=fq1.split('\n')
fq2 = open(pathn+'赋权2.txt',encoding='gbk',errors='replace')
fq2=fq2.read()
keyword2=fq2.split('\n')

word_num = open(pathn+"word_num.txt", "w+",encoding='gbk',errors='replace') 
word_num.write('prov_name'+' '+'year'+' '+'word_num1'+' '+'word_num2'+' '+'word_num3'+' '+'word_num4'+' '+'keynum1'+' '+'keynum2'+'\n')

files= os.listdir(path)  #得到文件夹下的所有文件名称
for file in files: 
    if int(file[-8:-4])<2008:               #苹果和联想两台电脑处理的数据，编码不一样
        f = open(path+file,encoding='gbk',errors='replace') 
    else:
        f = open(path+file,encoding='utf-8',errors='replace')  
    try:
        text=f.read()
        text=text.replace('\n','')
        text=text.replace(' ','')
        wordn1=len(text)                    #直接进行字数统计
        g = re.split(r"[，。？；,.]", text)   #根据文章中的实际情况添加分割句子的标点
        wordn2=len(g)-1 
        wordn3=len(g)-1                    #如果从0开始计数，则会少了一些标点符号
        wordn4=len(g)-1
        keynum1=0
        keynum2=0
        for line in g:
            line=line.strip()                #删除空格
            wordn2=wordn2+len(line)          #逐行统计字数，与全文直接统计字数，略有差异
            a=0
            for k in keyword1:
                if re.match('.*'+k+'.*',line): 
                    wordn3=wordn3+len(line)*1   #额外增加0.5倍数
                    keynum1=keynum1+1
                else:
                    continue
            for k2 in keyword2:
                if re.match('.*'+k2+'.*',line): 
                    wordn4=wordn3-len(line)*0.5  #额外减少0.5倍数
                    keynum2=keynum2+1
                else:
                    continue
            wordn3=wordn3+len(line)
            wordn4=wordn4+len(line)
    except:
        print(file)   #如果有没能成功读出的文档，则显示是那一篇（没有这种情形）
    word_num.write(file[0:-8]+' '+file[-8:-4]+' '+str(wordn1)+' '+str(wordn2)+' '+str(wordn3)+' '+str(wordn4)+' '+str(keynum1)+' '+str(keynum2)+'\n')
    f.close()
word_num.close()

In [ ]:
#本讲结束，下一将：网络爬虫入门